# SEQUENCE-TO-SEQUENCE

Sequence-to-sequence learning using neural networks was first introduced in 2014 by Ilya Sutskever, Oriol Vinyals and Quoc Le in their paper:  
<i>[1] "Sequence to Sequence Learning with Neural Networks"</i>.

The most common sequence-to-sequence models are <i>encoder-decored</i> models, which commonly use a recurrent neural network to encode the source sentence into a signle vector <i>(context vector)</i>. We can think of the context vector as being an abstract representation of the entire input sentence. This vector is then decoded by a second RNN which learns to output the target sentence by generating it one word at a time.

![Seq2Seq](img/seq2seq.png "Seq2Seq")

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

from src.seq2seq import Seq2Seq
from src.layers import affine_forward, affine_backward, cross_entropy_loss, temporal_cross_entropy_loss
from src.layers import word_embedding_forward, lstm_forward
from src.layers import attention_forward, attention_backward
from src.layers import temporal_attention_forward, temporal_attention_backward
from src.utils.gradient_check import eval_numerical_gradient, rel_error

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

# plot configuration
%matplotlib inline
plt.rcParams["figure.figsize"] = (15.0, 12.0) # set default size of plots
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"

Consider the task of translating a sentence from one language to another. The source sentence is passed through an embedding layer and then input through the encoder. Once the final word has been passed into the RNN via the embedding layer, we use the final hidden state as the context vector and we can start decoding it to get the target sentence.  
The words in the decoder are generated one after another. We always use the `<START>` token for the first input to the decoder. During training, for subsequent inputs we will use the actual ground truth next word in the sequence and we stop generating words once we generate the target length. Once we have our predicted target sentence, we compare it against our actual target sentence to calculate the loss.  
During inference, every predicted word by the decoder we will feed as the next input and we keep generating words until the model outputs the `<END>` token or after a certain amount of words have been generated.

![Seq2Seq Multilayer](img/seq2seq_multilayer.png "Seq2Seq Multilayer")

Rare words are replaced with a special `<UNK>` token.  
In addition, since we are training with minibatches containing sequences of different lengths, shorter sequences are padded with a special `<NULL>` token after the `<END>` token. This is done so that all sequences can be extended to be the same length (the length of the longest sequence). In this way the minibatch examples have the shape of a tensor. Loss and gradients will <b>not</b> be computed for the `<NULL>` tokens.

In [2]:
# Initialize toy example to check the implementation.
np.random.seed(13)

batch_size = 2
src_seq_len = 4
src_vocab_size = 10
src_embed_dim = 4

trg_seq_len = 7
trg_vocab_size = 10
trg_embed_dim = 4

hidden_dim = 5

null_idx = 0
start_idx = 1
end_idx = 2

src = np.random.randint(low=0, high=src_vocab_size, size=(batch_size, src_seq_len))
trg = np.random.randint(low=0, high=trg_vocab_size, size=(batch_size, trg_seq_len + 1))

print("Example source:\n", src)
print("Example target:\n", trg)

Example source:
 [[2 0 0 6]
 [2 4 9 3]]
Example target:
 [[4 2 6 5 9 4 2 0]
 [3 5 3 6 5 1 2 8]]


In [3]:
# Check the backward pass for the Seq2Seq model.
seq2seq_model = Seq2Seq(src_seq_len, src_vocab_size, src_embed_dim,
                        trg_seq_len, trg_vocab_size, trg_embed_dim,
                        hidden_dim, null_idx, start_idx, end_idx,
                        n_layers=2, cell_type="lstm", dtype=np.float64)

loss, grads = seq2seq_model.loss(src, trg)
f = lambda _ : seq2seq_model.loss(src, trg)[0]

for param_name in sorted(grads):
    param_grad_num = eval_numerical_gradient(f, seq2seq_model.params[param_name], verbose=False, h=1e-6)
    print("%s relative error: %e" % (param_name, rel_error(param_grad_num, grads[param_name])))

W_embed_dec relative error: 1.050069e-06
W_embed_enc relative error: 5.850212e-07
W_out_dec relative error: 1.162662e-06
Wh_0_dec relative error: 2.078131e-05
Wh_0_enc relative error: 6.126555e-04
Wh_1_dec relative error: 2.453824e-05
Wh_1_enc relative error: 3.307894e-05
Wx_0_dec relative error: 3.661329e-05
Wx_0_enc relative error: 4.405918e-05
Wx_1_dec relative error: 5.841204e-05
Wx_1_enc relative error: 7.050007e-05
b_0_dec relative error: 1.714063e-06
b_0_enc relative error: 2.694240e-06
b_1_dec relative error: 1.922607e-06
b_1_enc relative error: 9.101198e-07
b_out_dec relative error: 8.665735e-09


# ATTENTION

Learning with attention was introduced by Dzmitry Bahdanau, KyungHyun Cho and Yoshua Bengio in 2014 in their paper:  
<i>[2] "Neural Machine Translation by Jointly Learning to Align and Translate"</i>.

Using the final hidden state of the encoder as the single "context vector" for sequence-to-sequence models is a bottleneck in improving the performance. Learning with attention allows a model to automatically soft-search for parts of a source sentence that are relevant to predicting a target word.

![Seq2Seq Attention](img/seq2seq_attention.png "Seq2Seq Attention")

The attention mechanism is employed by the decoder. Words generated by the decoder are inferenced based not only on the current hidden state $ h_{t} $, but also on an attention output $ c_{t} $, and the attention output is recalculated at every time-step.  
First, we calculate the attention scores, $ e_{t} $, at time-step $ t $ by computing a dot-product between the current hidden state of the decoder $ h_{t} $ and every hidden state of the encoder $ h_{j}^{enc} $:

$$ e_{t, j} = h_{t}.h_{j}^{enc} $$

The attention scores tell us how well the input at position $ j $ matches with the output at position $ t $.  
After that we compute the attention distribution using a softmax function:

$$ \displaystyle a_{t, j} = \frac{ e^{e_{t, j}}}{\displaystyle \sum_{k=1} e^{e_{t, k}}} $$

And finally, we compute the attention output using a weighted sum over the hidden states of the encoder:

$$ \displaystyle c_{t} = \sum_{j = 1} a_{t, j} h_{j}^{enc} $$

This attention output is concatenated to the decoder hidden state and the new vector $ [h_{t}, a_{t}]$ is then used to generate the next output.

The authors of [1] state that they found it extremely helpful to reverse the order of the words of the input sequence because this way it is easier for SGD to "establish communication" between the input and the output. This observation coupled with the attention mechanism leads to the use of a bidirectional RNN for the encoder.  
A bidirectional RNN consists of a forward and a backward RNN resulting in a sequence of <i>forward hidden states</i> and <i>backward hidden states</i>. Concatenating the forward hidden state $ \stackrel{\rightarrow}{h}_{j} $ and the backward hidden state $ \stackrel{\leftarrow}{h}_{j} $ we obtain an annotation vector $ h_{j} = [\stackrel{\rightarrow}{h}_{j}, \stackrel{\leftarrow}{h}_{j}] $. We can think of the annotation vector as containing summaries of both the preceding words and the following words around word $ j $. This sequence of annotations is later used by the attention model to compute the attention scores.

![Seq2Seq Bidirectional](img/seq2seq_bidirectional.png "Seq2Seq Bidirectional")

A more general definition of attention is the following:  
Given a set of vector <i>values</i>, and a vector <i>query</i>, <b>attention</b> is a technique to compute a weighted sum of the values, dependet on the query. In our case, each decoder hidden state is the query, and the encoder hidden states are the values.

In [4]:
N, T = src.shape
H = seq2seq_model.hidden_dim
n_layers = seq2seq_model.n_layers

# Compute the embeddings
src_embeds, _ = word_embedding_forward(src, seq2seq_model.params["W_embed_enc"])

# Run through the encoder.
z = np.ndarray((N, n_layers, H))
h0 = np.zeros((N, H))
h_enc = src_embeds
for i in range(n_layers):
    h_enc, _ = lstm_forward(h_enc, h0, seq2seq_model.params["Wx_%d_enc" % i],
                                       seq2seq_model.params["Wh_%d_enc" % i],
                                       seq2seq_model.params["b_%d_enc" % i])
    z[:, i, :] = h_enc[:, -1, :]

# Compute the embeddings of the target words
trg_embeds, _ = word_embedding_forward(trg[:, :-1], seq2seq_model.params["W_embed_dec"])

# Run through the decoder
h_dec = trg_embeds
for i in range(n_layers):
    h_dec, _ = lstm_forward(h_dec, h0, seq2seq_model.params["Wx_%d_dec" % i],
                                       seq2seq_model.params["Wh_%d_dec" % i],
                                       seq2seq_model.params["b_%d_dec" % i])

print("check attention_backward")
h_t = h_dec[:, 0]
y = np.random.randint(low=0, high=(src_seq_len - 1), size=batch_size)

att_out, cache = attention_forward(h_t, h_enc)
loss, dout = cross_entropy_loss(att_out, y)
dh_t, dh_enc = attention_backward(dout, cache)

f = lambda _: cross_entropy_loss(attention_forward(h_t, h_enc)[0], y)[0]
dh_t_num = eval_numerical_gradient(f, h_t, verbose=False, h=1e-6)
print("h_t relative error: %e" % (rel_error(dh_t, dh_t_num)))

dh_enc_num = eval_numerical_gradient(f, h_enc, verbose=False, h=1e-6)
print("h_enc relative error: %e" % (rel_error(dh_enc, dh_enc_num)))


print("\ncheck temporal_attention_backward")
y = np.random.randint(low=0, high=(src_seq_len - 1), size=(batch_size, trg_seq_len))

att_out, cache = temporal_attention_forward(h_dec, h_enc)
loss, dout = temporal_cross_entropy_loss(att_out, y)
dh_dec, dh_enc = temporal_attention_backward(dout, cache)

f = lambda _: temporal_cross_entropy_loss(temporal_attention_forward(h_dec, h_enc)[0], y)[0]
dh_dec_num = eval_numerical_gradient(f, h_dec, verbose=False, h=1e-6)
print("h_t relative error: %e" % (rel_error(dh_dec, dh_dec_num)))

dh_enc_num = eval_numerical_gradient(f, h_enc, verbose=False, h=1e-6)
print("h_enc relative error: %e" % (rel_error(dh_enc, dh_enc_num)))

check attention_backward
h_t relative error: 8.298343e-07
h_enc relative error: 2.545957e-09

check temporal_attention_backward
h_t relative error: 5.514161e-06
h_enc relative error: 1.362537e-08


In [5]:
# Check the backward pass for the Seq2Seq model with attention.
attention_model = Seq2Seq(src_seq_len, src_vocab_size, src_embed_dim,
                          trg_seq_len, trg_vocab_size, trg_embed_dim,
                          hidden_dim, null_idx, start_idx, end_idx,
                          attention=True, n_layers=2, cell_type="lstm", dtype=np.float64)

loss, grads = attention_model.loss(src, trg)
f = lambda _ : attention_model.loss(src, trg)[0]

for param_name in sorted(grads):
    param_grad_num = eval_numerical_gradient(f, attention_model.params[param_name], verbose=False, h=1e-6)
    print("%s relative error: %e" % (param_name, rel_error(param_grad_num, grads[param_name])))

W_embed_dec relative error: 3.751720e-07
W_embed_enc relative error: 1.196485e-06
W_out_dec relative error: 4.294001e-06
Wh_0_dec relative error: 1.064747e-03
Wh_0_enc relative error: 1.214916e-03
Wh_1_dec relative error: 2.852304e-02
Wh_1_enc relative error: 1.604084e-03
Wx_0_dec relative error: 4.231066e-05
Wx_0_enc relative error: 7.019910e-05
Wx_1_dec relative error: 1.129603e-04
Wx_1_enc relative error: 4.273549e-04
b_0_dec relative error: 1.671579e-06
b_0_enc relative error: 6.479042e-06
b_1_dec relative error: 4.169366e-07
b_1_enc relative error: 3.501972e-06
b_out_dec relative error: 7.253745e-09
